In [1]:
import pandas as pd
import os

ROOT = os.pardir
train_raw_data = pd.read_csv(os.path.join(ROOT, "data/train.csv"))
test_raw_data = pd.read_csv(os.path.join(ROOT, "data/test.csv"))

In [2]:
train_raw_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
test_raw_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Text Cleaning

### 1. Lower case

In [4]:
train_raw_data["text"] = train_raw_data["text"].str.lower()
test_raw_data["text"] = test_raw_data["text"].str.lower()

train_raw_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1


### 2. Remove URLs

In [5]:
import text_cleaning

In [6]:
train_raw_data[["id", "text"]].query("id in [5557, 3619, 5961, 3636, 9156]").values

array([[3619,
        'did josephus get it wrong about antiochus epiphanes and the abomination of desolation? read more: http://t.co/fwj9ccyw6k'],
       [3636,
        'emotional desolation the effect of alcoholism/addiction on family - http://t.co/31tgtlz3ya forgiving is hard http://t.co/c7rco2emwf'],
       [5557,
        "100  1' mix new flat double sided linerless bottle caps you choose mix flattened - full re\x89û_ http://t.co/w00kjprfdr http://t.co/mixl1pfrje"],
       [5961,
        'freak accident? sure. looking for someone to blame? maybe. remember that player broke his leg cuz cart was at back of end zone? common sense'],
       [9156,
        'suicide bomber kills 15 in saudi security site mosque http://t.co/yytkp1z5kg via @reuters']],
      dtype=object)

In [7]:
url_pattern = r"(\w+:\/\/\S+)"
train_raw_data["text"] = train_raw_data["text"].apply(text_cleaning.remove_URL, url_pattern=url_pattern)
test_raw_data["text"] = test_raw_data["text"].apply(text_cleaning.remove_URL, url_pattern=url_pattern)

train_raw_data[["id", "text"]].query("id in [5557, 3619, 5961, 3636, 9156]").values

array([[3619,
        'did josephus get it wrong about antiochus epiphanes and the abomination of desolation? read more:  '],
       [3636,
        'emotional desolation the effect of alcoholism/addiction on family -   forgiving is hard  '],
       [5557,
        "100  1' mix new flat double sided linerless bottle caps you choose mix flattened - full re\x89û_    "],
       [5961,
        'freak accident? sure. looking for someone to blame? maybe. remember that player broke his leg cuz cart was at back of end zone? common sense'],
       [9156,
        'suicide bomber kills 15 in saudi security site mosque   via @reuters']],
      dtype=object)

### 3. Handle HTML

In [8]:
train_raw_data["text"] = train_raw_data["text"].apply(text_cleaning.decode_html)
test_raw_data["text"] = test_raw_data["text"].apply(text_cleaning.decode_html)

train_raw_data["text"].sample(5).values

/home/nj3/miniconda3/envs/nlp_env/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


array(['young children among those rescued from capsized boat off libya   via @irishtimesworld',
       'the more i listen to it the more i believe casualties of cool is one of the best albums there ever was.',
       "map: typhoon soudelor's predicted path as it approaches taiwan; expected to make landfall over southern china by s\x89û_  ",
       'severe storm weakening as it moves se towards lubbock area.  outflow boundary may create dust and 50 mph gusts  ',
       "when your body's like 'go to fuck to sleep sami' and your mind's like 'make an emergency plan for every natural disaster go'"],
      dtype=object)

### 4. Remove Twitter handles